* Pytorch에서는 주요 모델을 제공하고 있으며, 이를 이용해서 손쉽게 모델을 학습할 수 있습니다. 

* https://pytorch.org/vision/0.15/models.html#classification

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

In [2]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [11]:
resnet = models.resnet18(weights=None, num_classes=10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet.to(device)

epochs = 30

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = resnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

[1,   100] loss: 1.703
[1,   200] loss: 1.383
[1,   300] loss: 1.285
[2,   100] loss: 1.041
[2,   200] loss: 1.010
[2,   300] loss: 0.972
[3,   100] loss: 0.817
[3,   200] loss: 0.812
[3,   300] loss: 0.814
[4,   100] loss: 0.677
[4,   200] loss: 0.704
[4,   300] loss: 0.697
[5,   100] loss: 0.564
[5,   200] loss: 0.601
[5,   300] loss: 0.586
[6,   100] loss: 0.465
[6,   200] loss: 0.512
[6,   300] loss: 0.498
[7,   100] loss: 0.382
[7,   200] loss: 0.426
[7,   300] loss: 0.431
[8,   100] loss: 0.305
[8,   200] loss: 0.340
[8,   300] loss: 0.356
[9,   100] loss: 0.234
[9,   200] loss: 0.286
[9,   300] loss: 0.288
[10,   100] loss: 0.183
[10,   200] loss: 0.239
[10,   300] loss: 0.246
[11,   100] loss: 0.150
[11,   200] loss: 0.185
[11,   300] loss: 0.188
[12,   100] loss: 0.124
[12,   200] loss: 0.133
[12,   300] loss: 0.182
[13,   100] loss: 0.118
[13,   200] loss: 0.125
[13,   300] loss: 0.147
[14,   100] loss: 0.088
[14,   200] loss: 0.109
[14,   300] loss: 0.126
[15,   100] loss: 0

모델을 평가해봅시다

In [25]:
correct = 0
total = 0
with torch.no_grad():
    for img, label in testloader:
        img = img.to(device)
        label = label.to(device)
        outputs = resnet(img)
        predicted = torch.argmax(outputs.data, 1)
        print('pred', predicted)
        print('label',label)
        correct += (predicted == label).sum().item()
        total += label.size(0)

pred tensor([3, 8, 0, 0, 4, 6, 1, 6, 3, 1, 3, 9, 4, 7, 9, 8, 5, 7, 8, 6, 7, 2, 4, 9,
        4, 2, 4, 0, 9, 6, 3, 5, 4, 5, 9, 8, 4, 1, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9,
        7, 6, 9, 8, 6, 5, 8, 8, 5, 9, 5, 4, 7, 5, 4, 9, 3, 2, 1, 2, 3, 7, 2, 6,
        8, 8, 0, 2, 2, 3, 3, 8, 8, 0, 1, 7, 2, 7, 2, 9, 8, 9, 0, 3, 8, 6, 4, 6,
        6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 5, 0, 6, 2, 0, 3, 0, 4, 3, 5,
        8, 3, 1, 2, 8, 2, 0, 3], device='cuda:0')
label tensor([3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9,
        5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 2, 4, 1, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9,
        7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 4, 6, 7, 3, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6,
        8, 8, 0, 2, 9, 3, 3, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6,
        6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7,
        8, 3, 1, 2, 8, 0, 8, 3], device='cuda:0')
pred tensor([6, 0, 4, 1, 8, 9, 1, 2, 9, 7, 2, 0, 3, 5, 6, 3, 8, 8, 6, 6, 5, 2, 8, 9,
    

In [26]:
print(correct/total)

0.757


* loss는 pytorch의 Tensor이며, loss.items()는 float입니다. 

In [5]:
loss

tensor(0.0327, device='cuda:0', grad_fn=<NllLossBackward0>)

In [7]:
type(loss)

torch.Tensor

In [6]:
loss.item()

0.03270659223198891

In [8]:
type(loss.item())

float

In [13]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
from torchsummary import summary

summary(resnet, (3, 32, 32))  

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13             [-1, 64, 8, 8]             128
             ReLU-14             [-1, 6